#Web Scraping


In [151]:
import requests
import json
from bs4 import BeautifulSoup


**It takes at least 1M characters to train a model based on RNN ~ 3.500 comments** 

In [152]:
page_url = 'https://www.pudelek.pl/internauci-biora-pod-lupe-zachowanie-meghan-markle-pod-zamkiem-w-windsorze-przyklad-klasy-i-godnosci-vs-ta-kobieta-nie-ma-wstydu-6810969060067968a/'
page = requests.get(page_url)

In [153]:
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

In [154]:
clas = "sc-1mskw74-0 sc-7eqdwf-0 sc-q1w81m-0 cyNurK"
comm = soup.find_all("div", {"class": clas})
print(comm[4].get_text())
print(len(comm))

Megan czuje sie zagubiona w UK,dlatego Tak kurczowo trzyma sie meza, ona gra pierwsze skrzypce w USA gdzie Harry jest Sam I Nie ma za wiele do powiedzenia
19


**A HTML file does not contain all comments, padding is inteactive and does not chcenge the url adress, so there is a need to use API through GraphQL.**

In [184]:
query = """query comments(
    $productId: String!
      $materialId: String!
        $parentId: String
          $offset: Int = 0
            $first: Int = 10
              $sort: CommentsSortType = LATEST

                  ) {
                      comments: comments(
                            productId: $productId
                                contentId: $materialId
                                    parentId: $parentId
                                        offset: $offset
                                            first: $first
                                                sort: $sort
                                                  ) {
                                                        ...commentEdges
                                                            pageInfo {
                                                                    hasNextPage
                                                                          hasPreviousPage
                                                                                endCursor
                                                                                      totalCount
                                                                                              }
                                                                                                 }

                                                                                                      }
                                                                                                                                  
                                                                                                   fragment commentEdges on CommentsConnection {
                                                                                                         edges {
                                                                                                             node {
                                                                                                                      text
                                                                                                                          nick
                                                                                                                               id
                                                                                                                                   replies
                                                                                                                                       }
                                                                                                                                          }
                                                                                                                                             }"""
#query return max 100 comments
main_comm = []
for off in range(0, 1100, 100):
  data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6810969060067968", "offset": off, "first": 100, "sort": "LATEST"}}
  response = requests.post('https://data-api.wp.pl/graphql', json=data)
  main_comm.append(response.json())
#json.dumps(response.json(), indent=2, ensure_ascii=False)
#print(str(main_comm[1])[-500:])

In [197]:
#Dict to str
comments=''
for mc in main_comm:
    for i in range(0, len(mc['data']['comments']['edges'])):
      a = mc['data']['comments']['edges'][i]['node']

      text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
      nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

      comments = comments + text + nick

In [ ]:
#print(comments[:500])

In [200]:
#number of main comments
comments.count('nick')

576

In [198]:
len(comments)

115975

**Getting replies to main comments**

In [204]:
#getting id when replies > 0
parentId = [] 
for mc in main_comm:
  for i in range(0, len(mc['data']['comments']['edges'])):
    if mc['data']['comments']['edges'][i]['node']['replies']>0:
      parentId.append(mc['data']['comments']['edges'][i]['node']['id'])

In [208]:
for id in parentId:
  data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6810969060067968", "parentId": id, "offset": 0, "first": 100, "sort": "LATEST"}}
  response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
  replies = response_rep.json()

  for i in range(0, len(replies['data']['comments']['edges'])):
    a = replies['data']['comments']['edges'][i]['node']

    text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
    nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

    comments = comments + text + nick

In [211]:
#another round, if any main comment had more than 100 replies
for id in parentId:
  data = {"query": query, "variables": {"productId": "5980322589471361", "materialId": "6810969060067968", "parentId": id, "offset": 100, "first": 100, "sort": "LATEST"}}
  response_rep = requests.post('https://data-api.wp.pl/graphql', json=data)
  replies = response_rep.json()

  for i in range(0, len(replies['data']['comments']['edges'])):
    a = replies['data']['comments']['edges'][i]['node']

    text = str(a).split("{")[1].split(", \'nick\'")[0] + '\n'
    nick = '\'nick\'' + str(a).split("{")[1].split(", \'nick\'")[1].split(', \'id\'')[0] + '\n' + '\n'

    comments = comments + text + nick

In [212]:
len(comments)

240678

In [ ]:
#print(comments[-500:])

In [213]:
comments.count('nick')

1327

In [214]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks


In [215]:
# text_file = open("komentarze.txt", "w")
# n = text_file.write(comments)
# text_file.close()